1. PREPROCESS

In [1]:
%run setup_imports.py
import os
import pandas as pd

Preprocess module imported successfully!


In [ ]:
# Preprocessing input data
input_folder = r'C:\PRIM\dataset\inputs'
output_folder = r'C:\PRIM\dataset\inputs_preprocessed'

os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith(".xml"):  
        input_xml_path = os.path.join(input_folder, file_name)

        print(f"Processing: {input_xml_path}")

        # Preprocess the file
        tree = preprocess.load_xml(input_xml_path)
        extracted_data = preprocess.extract_data_from_input(tree)
        df = preprocess.data_to_dataframe(extracted_data)
        df_interpolated = preprocess.interpolate_data_with_flag(df)
        df_normalized = preprocess.normalize(df_interpolated)
        output_csv_path = os.path.join(output_folder, file_name.replace(".xml", "_preprocessed.csv"))
        preprocess.save_data(df_normalized, output_csv_path)

        print(f"Saved preprocessed file to: {output_csv_path}")


In [ ]:
# Preprocessing output data

input_folder = r'C:\PRIM\dataset\outputs'
output_folder = r'C:\PRIM\dataset\outputs_preprocessed'
os.makedirs(output_folder, exist_ok=True)
desired_columns = [
    'frame', 'stroke', 'player 1', 'player 2',
    'backhand', 'forehand', 'serve', 'ball pass',
    'point', 'mistake', 'void serve'
]

for file_name in os.listdir(input_folder):
    if file_name.endswith(".xml"):  
        input_xml_path = os.path.join(input_folder, file_name)
        print(f"Processing: {input_xml_path}")
        tree = preprocess.load_xml(input_xml_path)
        extracted_data = preprocess.extract_data_from_output(tree)
        df = preprocess.data_to_dataframe(extracted_data)
        df['stroke'] = df['player 1'] + df['player 2']
        df['void serve'] = df['let serve'] + df['void serve']
        df = df[desired_columns]
        output_csv_path = os.path.join(output_folder, file_name.replace(".xml", "_preprocessed.csv"))
        preprocess.save_data(df, output_csv_path)
        print(f"Saved preprocessed file to: {output_csv_path}")
